In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
from uti.basic_train import *

In [3]:
train_ds,valid_ds = get_dataset(*get_data())
data = Databunch(*get_dl(train_ds,valid_ds,bs=64),c=10)
loss_func = F.cross_entropy

/home/jupyter/.fastai/data/mnist.pkl.gz


# Remove callback_handler 

Bringing everything inside learner. Start to have different code as Jeremy in fastai notebook

1. _set_learner
when callback created, it needs leaner information, such as model,opt...etc
which is only available when learner is created

However, leaner needs to add callbacks when Leaner is created

Previously we use callback_handler to act as intermediate interface to pass the values. 
But this can be done inside the Leaner, just do cb._set_learner(self) in fit loop

When inside fit loop, we have all the states ready, such as model, opt, epochs...
calling set_learner(self) inside fit will just pass Leaner itself into callbacks 

The set_learner here acts like constructor but it will only be triggered during run time. 

Pure software engineering, not ML involved 

But this way, we will have a very easy interface to use

2. Diff from Jeremy's code
I still kept Callback methods, which I think it is easier to have abstract class ready for later use (and possible maintance in the future)

I didn't use __getattr__ , explictly calling method name makes things clear, which attribute is from leaner, which is from callback itself

The rest of the ideas are the same as fastai 2019 part2 class

In [4]:
#export
class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [5]:
#export
#abstract class for new callback
class Callback():
    _order = 0
    def _set_learner(self,learn): self.learn = learn
    def begin_fit(self): pass
    def after_fit(self): pass
    def begin_epoch(self): pass
    def begin_validate(self): pass
    def after_epoch(self): pass
    def begin_batch(self): pass
    def after_batch(self): pass
    def after_loss(self): pass
    def begin_backward(self): pass
    def after_backward(self): pass
    def after_step(self): pass
    def after_cancel_train(self): pass   
    def after_cancel_epoch(self): pass
    def after_cancel_batch(self): pass
    

In [6]:
#export
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.n_epochs, self.n_iters = 0,0
    
    def begin_epoch(self):
        self.n_epochs = self.learn.epoch #current epoch
        self.learn.model.train()
        self.learn.in_train = True
        
    def after_batch(self):
        if not self.learn.in_train: return
        self.n_epochs += 1. / self.learn.iters #current epoch step, eg: 1.3 epochs
        self.n_iters +=1
        
    def begin_validate(self):
        self.learn.model.eval()
        self.learn.in_train = False

In [7]:
#export
class NewLearner():
    def __init__(self, model,opt,loss_func,data,cbs=None):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data
        self.in_train = False
        self.cbs = []
        self.cbs.append(TrainEvalCallback())
        if cbs:
            for cb in sorted(cbs,key = lambda x: x._order):
                self.cbs.append(cb)
        
    def one_batch(self,xb,yb):
        try: 
            self.xb, self.yb = xb, yb
            for cb in self.cbs: cb.begin_batch()
            self.preds = self.model(self.xb)
            self.loss = self.loss_func(self.preds,self.yb)
            for cb in self.cbs: cb.after_loss()
            
            if not self.in_train: return
            
            for cb in self.cbs: cb.begin_backward()
            self.loss.backward()
            for cb in self.cbs: cb.after_backward()
            self.opt.step()
            for cb in self.cbs: cb.after_step()
            self.opt.zero_grad()
            
        except CancelBatchException: 
            for cb in self.cbs: cb.after_cancel_batch()
        
        finally:
            for cb in self.cbs: cb.after_batch()

    def all_batches(self):
        try:
            for xb,yb in self.dl:
                self.one_batch(xb,yb)
        except CancelEpochException: 
            for cb in self.cbs: cb.after_cancel_epoch()


    def fit(self,epochs):
        self.epochs = epochs
        self.iters = len(self.data.train_dl)
        try:
            for cb in self.cbs: cb._set_learner(self)
            for cb in self.cbs: cb.begin_fit()
            for epoch in range(epochs):
                self.epoch = epoch
                self.dl = self.data.train_dl
                for cb in self.cbs: cb.begin_epoch()
                self.all_batches()
        
                for cb in self.cbs: cb.begin_validate()
                self.dl = self.data.valid_dl
                with torch.no_grad(): self.all_batches()
                for cb in self.cbs: cb.after_epoch()
                
        except CancelTrainException: 
            for cb in self.cbs: cb.after_cancel_train()
                
        finally:
            for cb in self.cbs: cb.after_fit()
            #self.remove(cb)
            

In [8]:
learn = NewLearner(*get_model(data),loss_func,data)

In [9]:
learn.fit(1)

In [11]:
learn.cbs[0].n_epochs

0.9999999999999903

In [22]:
#export
class Accuracy(Callback):
    def begin_epoch(self):
        self.train_loss, self.valid_loss = 0, 0
        self.train_acc, self.valid_acc =0, 0
        self.train_bs_count = 0
        self.valid_bs_count = 0
        
    def accuracy(self, out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()
        
    def after_loss(self):
        with torch.no_grad():
            bs = self.learn.xb.shape[0]
            if self.learn.in_train: 
                self.train_loss += self.learn.loss * bs
                self.train_acc += self.accuracy(self.learn.preds,self.learn.yb) * bs
                self.train_bs_count += bs
            else: 
                self.valid_loss += self.learn.loss * bs
                self.valid_acc += self.accuracy(self.learn.preds,self.learn.yb) * bs
                self.valid_bs_count += bs
    
    def after_epoch(self):
        avg_train_loss = self.train_loss / self.train_bs_count
        avg_valid_loss = self.valid_loss / self.valid_bs_count
        avg_train_acc = self.train_acc / self.train_bs_count
        avg_valid_acc = self.valid_acc / self.valid_bs_count
        print(f'Train: {avg_train_loss}, {avg_train_acc}')
        print(f'Valid: {avg_valid_loss}, {avg_valid_acc}')
        print(' ')
        
            
        

In [23]:
learn = NewLearner(*get_model(data),loss_func,data,cbs=[Accuracy()])

In [24]:
learn.fit(3)

Train: 0.3097740709781647, 0.904699981212616
Valid: 0.4988924264907837, 0.8432999849319458
 
Train: 0.14133064448833466, 0.9574400186538696
Valid: 0.12808431684970856, 0.9609000086784363
 
Train: 0.10768666118383408, 0.9674400091171265
Valid: 0.10633409768342972, 0.9696999788284302
 


# Fin

In [1]:
from notebook2script import *

In [2]:
notebook2script('05b_new_learner.ipynb','newLeaner')

Converted 05b_new_learner.ipynb to uti/newLeaner_05b.py
